<a href="https://colab.research.google.com/github/abhinavdayal/EIP_Session5/blob/master/Davidnet_PersonAttributes_success01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [2]:
%tensorflow_version 2.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path
import time, math
import tensorflow as tf
from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


TensorFlow 2.x selected.


In [3]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [4]:
cat_df = pd.concat([
    df[["image_path"]]
], axis=1)
cat_df['gender'] = df.gender.astype('category').cat.codes
cat_df['imagequality'] = df.imagequality.astype('category').cat.codes
cat_df['age'] = df.age.astype('category').cat.codes
cat_df['weight'] = df.weight.astype('category').cat.codes
cat_df['carryingbag'] = df.carryingbag.astype('category').cat.codes
cat_df['footwear'] = df.footwear.astype('category').cat.codes
cat_df['emotion'] = df.emotion.astype('category').cat.codes
cat_df['bodypose'] = df.bodypose.astype('category').cat.codes
cat_df.head()

,image_path,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose
0,resized/1.jpg,1,0,2,0,1,2,2,1
1,resized/2.jpg,0,0,2,1,2,2,0,1
2,resized/3.jpg,1,2,3,0,1,0,2,1
3,resized/4.jpg,1,2,3,0,0,2,2,1
4,resized/5.jpg,0,2,2,2,2,0,2,1


In [5]:
num_units = {
    'gender': df.gender.astype('category').cat.categories,
    'imagequality': df.imagequality.astype('category').cat.categories,
    'age': df.age.astype('category').cat.categories,
    'weight': df.weight.astype('category').cat.categories,
    'carryingbag': df.carryingbag.astype('category').cat.categories,
    'footwear': df.footwear.astype('category').cat.categories,
    'emotion': df.emotion.astype('category').cat.categories,
    'bodypose': df.bodypose.astype('category').cat.categories
}

num_units

{'age': Index(['15-25', '25-35', '35-45', '45-55', '55+'], dtype='object'),
 'bodypose': Index(['Back', 'Front-Frontish', 'Side'], dtype='object'),
 'carryingbag': Index(['Daily/Office/Work Bag', 'Grocery/Home/Plastic Bag', 'None'], dtype='object'),
 'emotion': Index(['Angry/Serious', 'Happy', 'Neutral', 'Sad'], dtype='object'),
 'footwear': Index(['CantSee', 'Fancy', 'Normal'], dtype='object'),
 'gender': Index(['female', 'male'], dtype='object'),
 'imagequality': Index(['Average', 'Bad', 'Good'], dtype='object'),
 'weight': Index(['normal-healthy', 'over-weight', 'slightly-overweight', 'underweight'], dtype='object')}

In [6]:
len(num_units['age'])

5

In [7]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(cat_df, test_size=0.15, shuffle=True, random_state=40)
print(train_df.shape, val_df.shape)
len_train = train_df.shape[0]
len_test = val_df.shape[0] 

(11537, 9) (2036, 9)


In [8]:
train_df.head()

,image_path,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose
4661,resized/4662.jpg,0,2,1,0,0,1,1,1
3045,resized/3046.jpg,0,0,1,0,0,0,2,1
3404,resized/3405.jpg,0,0,1,0,2,2,2,0
6838,resized/6839.jpg,1,2,3,3,0,2,3,1
12231,resized/12233.jpg,0,2,1,0,0,2,2,0


In [9]:
# here we shall calculate incremental mean and variances so as to normalize data
# we are doing this otherwise the entire data will be loaded in memory at one time
# which can be huge. 
# NOTE: this need only be done once and then the values can be hardcoded.
from tqdm import tnrange, tqdm_notebook
n = 0
s = np.zeros(3)
sq = np.zeros(3)
y = None
for _, item in tqdm_notebook(train_df.iterrows()):
  x = cv2.imread(item["image_path"]).astype('double')
  s += np.sum(x, axis=(0,1))
  sq += np.sum(np.square(x), axis=(0,1))
  n += x.shape[0]*x.shape[1]
  y = x

mu = s/n
std = np.sqrt((sq/n - np.square(mu)))

print(mu, std, n)
#mu, std = (np.array([0.16208468, 0.16288063, 0.18275726]), np.array([0.25111935, 0.25353892, 0.27390219]))


[41.33479712 41.57640031 46.66141538] [63.9889454  64.6502506  69.85974087] 578880512


In [0]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, avg_color=np.array([0,0,0])):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        input_img[top:top + h, left:left + w, :] = avg_color

        return input_img

    return eraser

In [0]:
BATCH_SIZE = 32 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE =  0.01#@param {type:"number"}
WEIGHT_DECAY = 9e-4 #@param {type:"number"}
EPOCHS = 10 #@param {type:"integer"}

In [0]:
class PersonDataGenerator(tf.keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, train_mean, train_std, batch_size=BATCH_SIZE, shuffle=True, training=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.train_mean = train_mean
        self.train_std = train_std
        self.training = training
        self.eraser = get_random_eraser()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        
        images = []
        for image in [cv2.imread(item["image_path"]) for _, item in items.iterrows()]:
          # normalize
          image = ((image-self.train_mean)/self.train_std).astype('float32')

          # augment for training but not for test
          if self.training:
            image = self.eraser(image) # cutouts
            image = tf.image.random_flip_left_right(image) # horizontal flipping
            # TODO: try other types of augmentation as well
            
          
          images.append(image)

        target = np.stack(
            [items['gender'].values,
            items['imagequality'].values,
            items['age'].values,
            items['weight'].values,
            items['carryingbag'].values,
            items['footwear'].values,
            items['emotion'].values,
            items['bodypose'].values],
            axis = 0
        ).astype('int32')

        images = np.stack(images, axis=0)
        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, mu, std, batch_size=32)
test_gen = PersonDataGenerator(val_df, mu, std, batch_size=64, shuffle=False, training=False)

In [14]:
# get number of output units from data
images, targets = next(iter(train_gen))
print(images.shape, targets.shape)

(32, 224, 224, 3) (8, 32)


In [0]:
# initialize all weights between 0 and 1 of a 3x3xcxd filter uwing kiaming he policy
#https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  # how many filters
  fan = np.prod(shape[:-1])
  
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)

In [0]:
# creates a convolution layer with batch normalization and dropouts
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
    self.drop = tf.keras.layers.Dropout(0.05)

  def call(self, inputs, training=False):
    h = self.conv(inputs)
    if training:
      h = self.drop(h)
    return tf.nn.relu(self.bn(h))
    
  #def model(self):
  #    x = tf.keras.Input(shape=(224,224,3))
  #    return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [0]:
ConvBN(32).model().summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 32)      864       
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
tf_op_layer_Relu (TensorFlow [(None, 224, 224, 32)]    0         
Total params: 992
Trainable params: 928
Non-trainable params: 64
_________________________________________________________________


In [0]:
# This class creates a residual block (if res = true) else regular convolution layer
# It can apply pooling if passed. pool can be identity as well if no pooling is needed.
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h
    
  #def model(self):
  #    x = tf.keras.Input(shape=(56,56,3))
  #    return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [0]:
ResBlk(128, tf.keras.layers.MaxPooling2D(), True).model().summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
conv_bn_1 (ConvBN)              (None, 56, 56, 128)  3968        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 28, 28, 128)  0           conv_bn_1[0][0]                  
__________________________________________________________________________________________________
conv_bn_2 (ConvBN)              (None, 28, 28, 128)  147968      max_pooling2d[0][0]              
____________________________________________________________________________________________

In [0]:
ResBlk(128, tf.keras.layers.MaxPooling2D(), False).model().summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 56, 56, 3)]       0         
_________________________________________________________________
conv_bn_4 (ConvBN)           (None, 56, 56, 128)       3968      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 128)       0         
Total params: 3,968
Trainable params: 3,712
Non-trainable params: 256
_________________________________________________________________


In [0]:
# since david net is not a deep arcitecture, it is ok for it to have convolution layer in main line in every block.
# for deeper architectures there is problem of vanishing gradients otherwise.
class DavidNet(tf.keras.Model):
  def __init__(self, order, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.pool1 = tf.keras.layers.MaxPooling2D()
    self.pool2 = tf.keras.layers.MaxPooling2D()
    self.conv_bn1 = ConvBN(c)
    self.conv_bn2 = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.gpool = tf.keras.layers.GlobalMaxPool2D()
    self.order = order
    self.weight = weight
    self.out_layers = {}
    for key in self.order:
      self.out_layers[key] = tf.keras.layers.Dense(len(num_units[key]), name=f"{key}_output", kernel_initializer=init_pytorch, use_bias=False)
    """
    {'age': Index(['15-25', '25-35', '35-45', '45-55', '55+'], dtype='object'),
     'bodypose': Index(['Back', 'Front-Frontish', 'Side'], dtype='object'),
     'carryingbag': Index(['Daily/Office/Work Bag', 'Grocery/Home/Plastic Bag', 'None'], dtype='object'),
     'emotion': Index(['Angry/Serious', 'Happy', 'Neutral', 'Sad'], dtype='object'),
     'footwear': Index(['CantSee', 'Fancy', 'Normal'], dtype='object'),
     'gender': Index(['female', 'male'], dtype='object'),
     'imagequality': Index(['Average', 'Bad', 'Good'], dtype='object'),
     'weight': Index(['normal-healthy', 'over-weight', 'slightly-overweight', 'underweight'], dtype='object')}
    """

  def call(self, x, y):
    # inital convolution on inputs with 64 outputs of 32x32
    # followed by resnet block with 128 outpts of 16x16
    # followed by non-residual block with 256 outputs of 8x8
    # followed by residual block with 512 outputs of 4x4
    # followed by a global max pooling that is going to flatten the 5x5x512 channles to 1x512 picking only the max in each channel of 4x4
    # actually this happens for a batch so if batch size is 512 it will give 512x512 tensor
    
    h = self.pool1(self.conv_bn1(x)) # 128x128x64
    h = self.pool2(self.conv_bn2(h)) # 56x56x64
    h = self.gpool(self.blk3(self.blk2(self.blk1(h))))

    label_losses = []
    label_corrects = []
    i = 0
    for output in self.order:
      # dense layer with
      t = self.out_layers[output](h)*self.weight

      # calculate cross entropy
      ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=t, labels=y[i])

      # reduce the tensor by summing up the loss over entire batch. This is our overall loss
      label_losses = tf.concat([label_losses, tf.reshape(tf.reduce_sum(ce), [1])], 0)

      # calculate number of correct observations. This will help calcuate accuracy of training.
      label_corrects = tf.concat([label_corrects, tf.reshape(tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(t, axis = 1), y[i]), tf.float32)), [1])], 0)

      i += 1

    loss = tf.reduce_sum(label_losses)
    correct = tf.reduce_sum(label_corrects)
    
    return loss, correct, label_losses, label_corrects
  #def model(self):
  #      x = tf.keras.Input(shape=(224,224,3))
  #      return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [0]:
DavidNet().model().summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv_bn_5 (ConvBN)           (None, 224, 224, 8)       248       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 8)       0         
_________________________________________________________________
conv_bn_7 (ConvBN)           (None, 112, 112, 16)      1216      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
res_blk_2 (ResBlk)           (None, 28, 28, 128)       314880    
_________________________________________________________________
res_blk_3 (ResBlk)           (None, 14, 14, 256)       2959

In [0]:
order = cat_df.columns[1:]
model = DavidNet(order)
#model.model().summary()
#https://github.com/tensorflow/tensorflow/issues/25036

In [0]:
batches_per_epoch = len_train//BATCH_SIZE + 1

# interpolate the epoch probably linearly from 0 to max and then back to zero with
# steeper increase and gradual decrease
#lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, (EPOCHS+1)//4, (EPOCHS+1)//3, (EPOCHS+1)//2, EPOCHS], [0, LEARNING_RATE, LEARNING_RATE/2, LEARNING_RATE/4, LEARNING_RATE/8, 0])[0] 
lr_schedule = lambda t: 0.001 # fixed learning rate for now.

global_step = 0


# global_step/batches_per_epoch will be < 1 for first epoch per batch and between 1-2 for second and so on.
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)

# we also must optimize momentum in reverse direction of learning rate
# We are using nestrov momentum here.
opt = tf.keras.optimizers.SGD(lr_func, momentum=MOMENTUM, nesterov=True)
#opt = tf.compat.v1.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)

In [0]:
x, y = next(iter(train_gen))

In [25]:
loss, correct, batch_category_losses, batch_category_corrects = model(x, y)
print(loss)
print(correct)
print(batch_category_losses)
print(batch_category_corrects)

tf.Tensor(303.04282, shape=(), dtype=float32)
tf.Tensor(77.0, shape=(), dtype=float32)
tf.Tensor(
[22.150135 35.18192  51.477356 44.39227  35.155037 35.100006 44.449696
 35.13638 ], shape=(8,), dtype=float32)
tf.Tensor([20.  4. 12.  3. 16. 16.  1.  5.], shape=(8,), dtype=float32)


In [0]:
import datetime
def format(t):
  return str(datetime.timedelta(seconds=t))

In [30]:
t = time.time()
numlabels = len(order)
global_step = 0

# history recording for plotting curves
train_loss_history = {'total':[]}
test_loss_history = {'total':[]}
train_acc_history = {'total':[]}
test_acc_history = {'total':[]}

for key in num_units:
  train_loss_history[key] = []
  test_loss_history[key] = []
  train_acc_history[key] = []
  test_acc_history[key] = []


for epoch in tnrange(EPOCHS):
  train_loss = test_loss = train_correct = test_correct = 0.0
  category_train_loss = np.zeros(len(order))
  category_train_correct = np.zeros(len(order))
  category_test_loss = np.zeros(len(order))
  category_test_correct = np.zeros(len(order))

  tf.keras.backend.set_learning_phase(1)
  for (x, y) in tqdm_notebook(train_gen):
    with tf.GradientTape() as tape:
      # calculate the outputs for this batch
      loss, correct, batch_category_losses, batch_category_corrects = model(x, y)

    # Now we need to update the training variables of model using back propagation
    var = model.trainable_variables

    # calculate the gradients of loss vs the trainable_variables
    # tensorflow does it for us, differentiating loss against reach variable. Amazing!
    grads = tape.gradient(loss, var)

    # implement weight decay, but why multiply by batch size?
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE

    # apply the momentum. This updates the gradients for us
    opt.apply_gradients(zip(grads, var))

    global_step += 1

    train_loss += loss.numpy()
    train_correct += correct.numpy()
    category_train_loss += batch_category_losses.numpy()
    category_train_correct += batch_category_corrects.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in tqdm_notebook(test_gen):
    loss, correct, batch_category_losses, batch_category_corrects = model(x, y)
    test_loss += loss.numpy()
    test_correct += correct.numpy()
    category_test_loss += batch_category_losses.numpy()
    category_test_correct += batch_category_corrects.numpy()
  
  train_loss_history['total'].append(train_loss / (len_train*numlabels))
  test_loss_history['total'].append(test_loss / (len_test*numlabels))
  train_acc_history['total'].append(train_correct / (len_train*numlabels))
  test_acc_history['total'].append(test_correct / (len_test*numlabels))

  print(f"\nEPOCH:{epoch+1}, lr: {lr_schedule(epoch+1)}, train loss: {train_loss_history['total'][epoch]:.2f}, train acc: {train_acc_history['total'][epoch]:.2f}, val loss: {test_loss_history['total'][epoch]:.2f}, val acc: {test_acc_history['total'][epoch]:.2f}, time: {format(time.time() - t)}")
  for i in range(len(order)):
    k = order[i]
    train_loss_history[k].append(category_train_loss[i]/len_train)
    test_loss_history[k].append(category_test_loss[i]/len_test)
    train_acc_history[k].append(category_train_correct[i]/len_train)
    test_acc_history[k].append(category_test_correct[i]/len_test)
    print(f"{(k+'_output').upper()}: [TRAIN] loss: {train_loss_history[k][epoch]:.2f}, acc: {train_acc_history[k][epoch]:.2f}, [TEST] loss: {test_loss_history[k][epoch]:.2f}, acc: {test_acc_history[k][epoch]:.2f}")


EPOCH:1, lr: 0.001, train loss: 0.79, train acc: 0.67, val loss: 0.81, val acc: 0.63, time: 0:02:03.843247
GENDER_OUTPUT: [TRAIN] loss: 0.34, acc: 0.85, [TEST] loss: 0.36, acc: 0.82
IMAGEQUALITY_OUTPUT: [TRAIN] loss: 0.87, acc: 0.58, [TEST] loss: 0.91, acc: 0.55
AGE_OUTPUT: [TRAIN] loss: 1.32, acc: 0.42, [TEST] loss: 1.31, acc: 0.35
WEIGHT_OUTPUT: [TRAIN] loss: 0.93, acc: 0.64, [TEST] loss: 0.92, acc: 0.63
CARRYINGBAG_OUTPUT: [TRAIN] loss: 0.80, acc: 0.65, [TEST] loss: 0.91, acc: 0.58
FOOTWEAR_OUTPUT: [TRAIN] loss: 0.73, acc: 0.68, [TEST] loss: 0.75, acc: 0.66
EMOTION_OUTPUT: [TRAIN] loss: 0.86, acc: 0.71, [TEST] loss: 0.89, acc: 0.68
BODYPOSE_OUTPUT: [TRAIN] loss: 0.47, acc: 0.81, [TEST] loss: 0.46, acc: 0.80


KeyboardInterrupt: ignored

In [0]:
category_train_correct

[5598.0, 3730.0, 2330.0, 2951.0, 3696.0, 3834.0, 4094.0, 2837.0]